# 处理丢失数据

有两种丢失数据：
- None
- np.nan(NaN)

## 1. None

None是Python自带的，其类型为python object。因此，None不能参与到任何计算中。

object类型的运算要比int类型的运算慢得多  
计算不同数据类型求和时间  
%timeit np.arange(1e5,dtype=xxx).sum()

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [2]:
%timeit np.arange(1e5,dtype=np.int32)

51.9 µs ± 716 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [3]:
%timeit np.arange(1e5,dtype=object)

8.67 ms ± 627 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 2. np.nan（NaN）

np.nan是浮点类型，能参与到计算中。但计算的结果总是NaN。

但可以使用np.nan*()函数来计算nan，此时视nan为0。

In [4]:
arr1 = np.array([1,2,3,np.nan,4,5])
np.nansum(arr1)

15.0

In [5]:
arr1.sum()

nan

In [6]:
np.nan + 9

nan

## 3. pandas中的None与NaN

### 1) pandas中None与np.nan都视作np.nan

创建DataFrame

In [7]:
data = np.random.randint(0,100,size=(5,5))
columns = list('ABCDE')
df = DataFrame(data=data,columns=columns)
df

,A,B,C,D,E
0,86,28,20,42,23
1,37,71,87,68,69
2,40,10,61,25,13
3,64,46,4,72,71
4,46,4,84,64,6


使用DataFrame行索引与列索引修改DataFrame数据

In [10]:
# NaN  not a number
df.loc[2,'D'] = None
df.loc[3,'B'] = np.nan
df.loc[0,'E'] = np.nan
df

,A,B,C,D,E
0,86,28.0,20,42.0,NaN
1,37,71.0,87,68.0,69.0
2,40,10.0,61,NaN,13.0
3,64,NaN,4,72.0,71.0
4,46,4.0,84,64.0,6.0


In [12]:
# 虽然使用None进行赋值，但是DataFrame会把数据处理成np.nan
type(df.loc[2,'D'])

numpy.float64

练习：尝试对ndarray对象和Series对象分别赋值None和np.nan，观察两种类型对两种空值的处理结果

In [23]:
# 对于ndarray而言，如果数据类型是object，会把None作为None处理，np.nan作为np.nan处理
# 如果数据类型是float，会把None和np.nan都处理为np.nan
nd1 = np.array([1,2,3,4],dtype=np.float64)
nd1[0] = np.nan
nd1[1] = None
nd1

array([ nan,  nan,   3.,   4.])

In [20]:
# series对象会把None和np.nan都处理成np.nan，数据类型默认修改为float64
data = np.array([1,2,3,4,5])
s1 = Series(data)
s1[0] = None
s1[1] = np.nan
s1

0    NaN
1    NaN
2    3.0
3    4.0
4    5.0
dtype: float64

### 2) pandas中None与np.nan的操作

- ``isnull()``
- ``notnull()``
- ``dropna()``: 过滤丢失数据
- ``fillna()``: 填充丢失数据

(1)判断函数
- ``isnull()``
- ``notnull()``

In [28]:
df.isnull()

,A,B,C,D,E
0,False,False,False,False,True
1,False,False,False,False,False
2,False,False,False,True,False
3,False,True,False,False,False
4,False,False,False,False,False


In [29]:
# 配合any使用，可以查看所有行或列是否存在空值
# 可以控制axis来改变查看方向
df.isnull().any(axis=1)

0     True
1    False
2     True
3     True
4    False
dtype: bool

In [32]:
df.notnull()

,A,B,C,D,E
0,True,True,True,True,False
1,True,True,True,True,True
2,True,True,True,False,True
3,True,False,True,True,True
4,True,True,True,True,True


In [31]:
df.notnull().all(axis=1)

0    False
1     True
2    False
3    False
4     True
dtype: bool

(2) 过滤函数
- ``dropna()``

可以选择过滤的是行还是列（默认为行）

In [35]:
df.dropna(axis=1)

,A,C
0,86,20
1,37,87
2,40,61
3,64,4
4,46,84


也可以选择过滤的方式 how = 'all'

In [37]:
df.loc[0] = np.nan
df

,A,B,C,D,E
0,NaN,NaN,NaN,NaN,NaN
1,37.0,71.0,87.0,68.0,69.0
2,40.0,10.0,61.0,NaN,13.0
3,64.0,NaN,4.0,72.0,71.0
4,46.0,4.0,84.0,64.0,6.0


In [39]:
df.dropna(how='all')

,A,B,C,D,E
1,37.0,71.0,87.0,68.0,69.0
2,40.0,10.0,61.0,NaN,13.0
3,64.0,NaN,4.0,72.0,71.0
4,46.0,4.0,84.0,64.0,6.0


(3) 填充函数 Series/DataFrame
- ``fillna()``

In [40]:
df

,A,B,C,D,E
0,NaN,NaN,NaN,NaN,NaN
1,37.0,71.0,87.0,68.0,69.0
2,40.0,10.0,61.0,NaN,13.0
3,64.0,NaN,4.0,72.0,71.0
4,46.0,4.0,84.0,64.0,6.0


In [43]:
# 使用value参数设置要填充的值
filled_df = df.fillna(value=666)
filled_df

,A,B,C,D,E
0,666.0,666.0,666.0,666.0,666.0
1,37.0,71.0,87.0,68.0,69.0
2,40.0,10.0,61.0,666.0,13.0
3,64.0,666.0,4.0,72.0,71.0
4,46.0,4.0,84.0,64.0,6.0


可以选择前向填充还是后向填充

In [45]:
df.fillna(method='bfill')

,A,B,C,D,E
0,37.0,71.0,87.0,68.0,69.0
1,37.0,71.0,87.0,68.0,69.0
2,40.0,10.0,61.0,72.0,13.0
3,64.0,4.0,4.0,72.0,71.0
4,46.0,4.0,84.0,64.0,6.0


In [48]:
df.loc[2,'B':'D'] = np.nan
df

,A,B,C,D,E
0,NaN,NaN,NaN,NaN,NaN
1,37.0,71.0,87.0,68.0,69.0
2,40.0,NaN,NaN,NaN,13.0
3,64.0,NaN,4.0,72.0,71.0
4,46.0,4.0,84.0,64.0,6.0


In [52]:
# 使用limit来限定填充次数
# 注意：value参数是不能跟method参数共用的
df.fillna(method='ffill',axis=1,limit=2)

,A,B,C,D,E
0,NaN,NaN,NaN,NaN,NaN
1,37.0,71.0,87.0,68.0,69.0
2,40.0,40.0,40.0,NaN,13.0
3,64.0,64.0,4.0,72.0,71.0
4,46.0,4.0,84.0,64.0,6.0


对于DataFrame来说，还要选择填充的轴axis。记住，对于DataFrame来说：

- axis=0：index/行
- axis=1：columns/列

============================================

练习7：

1. 简述None与NaN的区别

2. 假设张三李四参加模拟考试，但张三因为突然想明白人生放弃了英语考试，因此记为None，请据此创建一个DataFrame,命名为ddd3

3. 老师决定根据用数学的分数填充张三的英语成绩，如何实现？
    用李四的英语成绩填充张三的英语成绩？

============================================

In [54]:
data = np.random.randint(0,150,size=(2,3))
index = ['张三','李四']
columns = ['语文','数学','英语']
df = DataFrame(data=data,index=index,columns=columns)
df.loc['张三','英语'] = None
df

,语文,数学,英语
张三,126,117,NaN
李四,73,124,33.0


In [57]:
math = df.loc['张三','数学']
df.fillna(math)

,语文,数学,英语
张三,126,117,117.0
李四,73,124,33.0


In [58]:
# 向前填充
df.fillna(method='ffill',axis=1)

,语文,数学,英语
张三,126.0,117.0,117.0
李四,73.0,124.0,33.0


In [59]:
# 向后填充
df.fillna(method='bfill')

,语文,数学,英语
张三,126,117,33.0
李四,73,124,33.0
